In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [1]:
import networkx as nx
import pandas as pd
import markov_clustering as mc
import community
import matplotlib.pyplot as plt
import numpy as np

import utils as ut

In [ ]:
def read_graph(file):
    
    G = nx.Graph()
    
    with open("../interactomes/"+file+".tsv", "r") as f:
        for row in f.readlines()[1:]:
            edge = row[:-1].split("\t")
            G.add_edge(edge[0],edge[1])
    
    return G


def find_best_inflation(mat):
    
    infl_lst = []
    for inflation in [i/10 for i in range(15, 26)]:
        
        result = mc.run_mcl(mat, inflation=inflation)
        clusters = mc.get_clusters(result)
        Q = mc.modularity(matrix=np.asmatrix(result), clusters=clusters)
        infl_lst.append((Q, inflation))
    
    return(max(infl_lst)[1])

def get_labels(G_lcc, clusters):
    
    # map node name to numbers
    lbls = {}
    for k in range(len(G_lcc)):
        lbls[k] = list(G_lcc.nodes())[k]

    # use labels
    for i in range(len(clusters)):
        clusters[i] = [lbls.get(item, item) for item in clusters[i]]
    
    return(clusters)


def MCL(G_lcc):
    
    mat = nx.to_numpy_matrix(G_lcc)
    
    # find best inflation
    max_infl = find_best_inflation(mat)
    
    result = mc.run_mcl(mat, inflation = max_infl)
    clusters = mc.get_clusters(result)
    
    # return name of genes
    return(get_labels(G_lcc, clusters))


def louvain(G_lcc):
    
    partition = community.best_partition(G_lcc)
    clusters = [] 
    for com in set(partition.values()) :
        list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
        clusters.append(list_nodes)

    return(clusters)


def hypergeom_test(mod, genes, G_lcc):
    
    M = len(G_lcc.nodes())
    n = len(genes)
    N = len(mod)
    x = len(set(genes).intersection(set(mod)))
    
    pval = hypergeom.cdf(x, M, n, N)
    
    return(pval, x, M, n, N)


def fill_df(df, clusters, G_lcc, genes, algo):
    r = df.shape[0]
    for idx, c in enumerate(clusters):
        df.loc[r+idx, 'cl_algo'] = algo
        df.loc[r+idx, 'mod_id'] = r + idx
        df.loc[r+idx, 'n_sg'] = ut.hypergeom_test(c, genes, G_lcc)[1]
        df.loc[r+idx, 'n_g'] = ut.hypergeom_test(c, genes, G_lcc)[4]
        df.loc[r+idx, 'sg_id'] = list(set(genes).intersection(set(c)))
        df.loc[r+idx, 'g_id'] = list(set(c))
        df.loc[r+idx, 'p_value'] = ut.hypergeom_test(c, genes, G_lcc)[0]
    
    return(df)


In [2]:
# Initialize the gene list
with open("seed_genes.txt","r") as f:
    genes = [gene.rstrip() for gene in f.readlines()]


### read data

ii = ut.read_graph("ii")
ui = ut.read_graph("ui")

# get LCC
G_ii = max(nx.connected_component_subgraphs(ii), key=len)
G_ui = max(nx.connected_component_subgraphs(ui), key=len)


### LOUVAIN 

cl_ii_lou = ut.louvain(G_ii)
cl_ui_lou = ut.louvain(G_ui)


### MCL

cl_ii_mcl = ut.MCL(G_ii)
cl_ui_mcl = ut.MCL(G_ui)


# create table
col_lst = ['cl_algo', 'mod_id', 'n_sg', 'n_g', 'sg_id', 'g_id', 'p_value']
df_ui = pd.DataFrame(data=None, columns = col_lst)
df_ii = pd.DataFrame(data=None, columns = col_lst)


### hypergeometric test - ui, MCL
df_ui = pd.DataFrame(data=None, columns = col_lst)
df_ui = ut.fill_df(df_ui, cl_ui_mcl, G_ui, genes, 'MCL')
       
        

### hypergeometric test - ii, MCL

df_ii = pd.DataFrame(data=None, columns = col_lst)
df_ii = ut.fill_df(df_ii, cl_ii_mcl, G_ii, genes, 'MCL')


### hypergeometric test - ui, Louvain

df_ui = ut.fill_df(df_ui, cl_ui_lou, G_ui, genes, 'Louvain')

### hypergeometric test - ii, Louvain

df_ii = ut.fill_df(df_ii, cl_ii_lou, G_ii, genes, 'Louvain')


#save

df_ui.to_csv('results/df_ui.csv')
df_ii.to_csv('results/df_ii.csv')


# get putative disease modules
#df_ui[(df_ui.n_g >= 10) & (df_ui.p_value < 0.05)]


In [3]:
df_ui

,cl_algo,mod_id,n_sg,n_g,sg_id,g_id,p_value
0,MCL,0,1,7,[TMSB4Y],"[POT1, ACTA1, ACTG1, TERF1, TERF2IP, TMSB4Y, E...",0.900957
1,MCL,1,1,5,[KDM5D],"[HIST3H3, KMT2A, AR, PCGF6, KDM5D]",0.805874
2,MCL,2,1,21,[SRY],"[Tcf12, SMAD3, Hhex, SRY, Dlx5, Egr2, EP300, K...",0.999345
3,MCL,3,1,23,[DDX3Y],"[SAP25, VHL, SMAD2, PCBP1, APBB1, NUDCD2, WWOX...",0.999695
4,MCL,4,1,14,[USP9Y],"[PMS1, ABRAXAS2, HS2ST1, MAPT, CLK3, NCKIPSD, ...",0.991357
5,MCL,5,1,7,[ZFY],"[ZFX, ZBTB9, ZFY, RNF2, L3MBTL2, CACNB3, NSD1]",0.900957
6,MCL,6,1,5,[TSPY1],"[EEF1A1, EEF1A2, CSNK2A1, HIST2H2BE, TSPY1]",0.805874
7,MCL,7,1,4,[CDY1],"[HIST2H2AC, CDY1, HIST1H4A, REV3L]",0.729246
8,MCL,8,2,15,"[RPS4Y2, RPS4Y1]","[IGSF8, RPS4Y2, HTR7, CD81, RPS4Y1, ZNF746, IC...",0.95481
9,MCL,9,1,21,[RBMY1A1],"[KHDRBS3, APOBEC3C, LNX1, IGF2BP1, AEN, PCDHB1...",0.999345


In [4]:
df_ii

,cl_algo,mod_id,n_sg,n_g,sg_id,g_id,p_value
0,MCL,0,1,6,[TMSB4Y],"[POT1, ACTA1, TERF1, TERF2IP, TMSB4Y, EWSR1]",0.970306
1,MCL,1,1,8,[SRY],"[EP300, KPNB1, SP1, HDAC3, SMAD3, SRY, AR, WDR5]",0.991391
2,MCL,2,1,16,[DDX3Y],"[SAP25, VHL, SMAD2, MCM2, ODF2, POC1A, FBXO6, ...",0.999959
3,MCL,3,1,11,[USP9Y],"[PMS1, HS2ST1, CLK3, UBC, NCKIPSD, USP9Y, VSX1...",0.998745
4,MCL,4,1,6,[ZFY],"[ZFX, ZBTB9, ZFY, RNF2, CACNB3, NSD1]",0.970306
5,MCL,5,2,20,"[RBMY1F, RBMY1A1]","[RBMY1F, PCDHB14, RASD1, YTHDC1, FAM103A1, APO...",0.999949
6,MCL,6,2,10,"[RPS4Y2, RPS4Y1]","[IGSF8, RPS4Y2, CD81, RPS4Y1, ZNF746, ICAM1, C...",0.975929
7,MCL,7,1,27,[TBL1Y],"[CCT7, TBL1XR1, CCT3, NCOR2, DNAJB1, NCOR1, TU...",1
8,Louvain,8,1,6,[TMSB4Y],"[POT1, ACTA1, TERF1, TERF2IP, TMSB4Y, EWSR1]",0.970306
9,Louvain,9,1,8,[SRY],"[EP300, KPNB1, SP1, HDAC3, SMAD3, SRY, AR, WDR5]",0.991391


## LCC for ii - LOUVAIN

In [ ]:
nx.draw(G_ii, node_size=3)
plt.show()

In [ ]:
#drawing
size = float(len(set(partition_ii.values())))
pos = nx.spring_layout(G_ii)
count = 0.
for com in set(partition_ii.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition_ii.keys()
                                if partition_ii[nodes] == com]
    nx.draw_networkx_nodes(G_ii, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G_ii, pos, alpha=0.5)
plt.show()

## LCC - ui - LOUVAIN

In [ ]:
# read data and create graph
ui = read_graph("ui")

# find LCC
G_ui = max(nx.connected_component_subgraphs(ui), key=len)

# community - LOUVAIN 
partition_ui = community.best_partition(G_ui)

In [ ]:
#drawing
size = float(len(set(partition_ui.values())))
pos = nx.spring_layout(G_ui)
count = 0.
for com in set(partition_ui.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition_ui.keys()
                                if partition_ui[nodes] == com]
    nx.draw_networkx_nodes(G_ui, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G_ui, pos, alpha=0.5)
plt.show()

## LCC - ii - MCL

In [ ]:
mc.draw_graph(ii_mat, clusters, node_size=50, with_labels=False, edge_color="silver")

## LCC - ui - MCL

In [ ]:
mc.draw_graph(ui_mat, clusters, node_size=50, with_labels=False, edge_color="silver")

## MCL - ui - test ipergeometrico

In [ ]:
####### test ipergeometrico sui seed genes!!!!

# x-1
# M -> population size
# n -> number of successes in the population
# N -> is the sample size 
# x -> numero seed genes nel cluster 

# https://github.com/GuyAllard/markov_clustering

# https://blog.alexlenail.me/understanding-and-implementing-the-hypergeometric-test-in-python-a7db688a7458

# https://www.biostars.org/p/66729/

# 